In [1]:
# imports
import folium
from haversine import haversine, Unit
from pprint import pprint
import numpy as np
import time as t
import pandas as pd
from datetime import datetime, timedelta
from decimal import Decimal, ROUND_UP
import osmnx as ox
import networkx as nx

ox.config(log_console=False, use_cache=True)

In [2]:
# Khi - HQ
Khi_latlng = (24.853803, 67.035395)

# Saleh Pat
slp_latlng = (27.519, 69.044)

# Hyd - Cantt
hyd_latlang = (25.399066, 68.359492)

# Bdn - Cantt
bdn_latlang = (24.657308, 68.859181)


In [3]:
# location where you want to find your route
place     = 'Sindh, Pakistan'
# find shortest route based on the mode of travel
mode      = 'drive'        # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'length'        # 'length','time'

# Get all nodes in a certain area which are permissible for driving
graph = ox.graph_from_place(place, network_type = mode, simplify = False)

C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [53]:
def zoom_path(path_list):
    
    for i in range(len(path_list)-1):
        
        new_val_x = (path_list[i][0] + path_list[i+1][0])/2
        new_val_y = (path_list[i][1] + path_list[i+1][1])/2
        new_val = (new_val_x, new_val_y)
        
        print('First value: ', path_list[i])
        print('New Mid Value: ', new_val)
        print('Second value: ', path_list[i+1])
        

In [54]:
def get_path_coordinates(cvy_start, cvy_dest):
    
    path_list = []
    if cvy_start == 'Karachi':
        # Khi - HQ
        start_latlng = Khi_latlng
        
        if cvy_dest == 'Hyderabad':
            end_latlng = hyd_latlang
        
        if cvy_dest == 'Badin':
            end_latlng = bdn_latlang
            
        if cvy_dest == 'Saleh Pat':
            end_latlng = slp_latlng
        
        
    if cvy_start == 'Hyderabad':
        # Khi - HQ
        start_latlng = hyd_latlang
        
        if cvy_dest == 'Karachi':
            end_latlng = Khi_latlng
        
        if cvy_dest == 'Badin':
            end_latlng = bdn_latlang
            
        if cvy_dest == 'Saleh Pat':
            end_latlng = slp_latlng
            
    if cvy_start == 'Badin':
        # Khi - HQ
        start_latlng = bdn_latlang
        
        if cvy_dest == 'Hyderabad':
            end_latlng = hyd_latlang
        
        if cvy_dest == 'Karachi':
            end_latlng = Khi_latlng
            
        if cvy_dest == 'Saleh Pat':
            end_latlng = slp_latlng
        
    if cvy_start == 'Saleh Pat':
        # Khi - HQ
        start_latlng = slp_latlng
        
        if cvy_dest == 'Hyderabad':
            end_latlng = hyd_latlang
        
        if cvy_dest == 'Badin':
            end_latlng = bdn_latlang
            
        if cvy_dest == 'Karachi':
            end_latlng = Khi_latlng

    
    # find the nearest node to the start location
    orig_node = ox.get_nearest_node(graph, start_latlng)

    # find the nearest node to the end location
    dest_node = ox.get_nearest_node(graph, end_latlng)
    
    #  find the shortest path
    shortest_route = nx.shortest_path(graph,
                                      orig_node,
                                      dest_node,
                                      weight=optimizer)
    
    for shortlisted_node in shortest_route:
        path_list.append((graph.nodes[shortlisted_node]["y"], 
                          graph.nodes[shortlisted_node]["x"]))
    
    path_list = zoom_path(path_list)
    
    return path_list

In [55]:
def get_input_distance(start_time, user_time, speed):
    
    t1 = datetime.strptime(str(start_time)[0:2] + ':' + str(start_time)[2:4], '%H:%M')
    t2 = datetime.strptime(str(user_time)[0:2] + ':' + str(user_time)[2:4], '%H:%M')
    td = t2 - t1
    time_spent = td.total_seconds()/3600
    dist_now = speed*time_spent
    
    return dist_now


In [56]:
def get_dist_coordinates(coordinates):
    
    cumulative_dist = 0
    dist_list = []
#     print(coordinates)
    
    for index in range(len(coordinates)):
        if index:
            dist_one = haversine(coordinates[index-1], coordinates[index])
            cumulative_dist += dist_one
            dist_list.append(cumulative_dist)
        else:
            dist_one = 0
            dist_list.append(dist_one)
    
    return dist_list


In [57]:
def get_relative_distance(user_distance, dist_coordinates):
    
    relative_dist = []
#     print(user_distance)
    
    for dist in dist_coordinates:
        
        temp = abs(dist - user_distance)
        relative_dist.append(temp)
        
    return relative_dist

In [58]:
def create_path(coordinates_cleaned, cvy_start, cvy_dest, cvy_color, folium_map):
# expects a list of tuples containing path coordinates ... [(1,1),(2,2),(3,3)...]
    
    path = coordinates_cleaned
    
    folium.PolyLine(path,
                color= cvy_color,
                weight=3,
                opacity=0.7).add_to(folium_map)

In [ ]:
"""Read data from excel file"""

# lst of column names which needs to be string
lst_str_cols = ['Start Time']
# use dictionary comprehension to make dict of dtypes
dict_dtypes = {x : 'str'  for x in lst_str_cols}

cvy_inputs = pd.read_excel('Inputs.xlsx', dtype=dict_dtypes)
cvy_inputs['Coordinates'] = cvy_inputs['Coordinates'].astype(object)
cvy_inputs['Distance'] = cvy_inputs['Distance'].astype(object)
cvy_inputs['Relative Distance'] = cvy_inputs['Relative Distance'].astype(object)
start_time = cvy_inputs['Start Time'].min()

for index, rows in cvy_inputs.iterrows():
    
    # get coordinates based on route and append to dataframe as new column 'Coordinates'
    path = get_path_coordinates(rows['Start'], rows['Destination'])
    # insert list of coordinates at a single index in dataframe
    cvy_inputs.at[index, 'Coordinates'] = path

C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


First value:  (24.8537419, 67.0354672)
New Mid Value:  (24.85334705, 67.0352235)
Second value:  (24.8529522, 67.0349798)
First value:  (24.8529522, 67.0349798)
New Mid Value:  (24.8528519, 67.03492130000001)
Second value:  (24.8527516, 67.0348628)
First value:  (24.8527516, 67.0348628)
New Mid Value:  (24.8527159, 67.034842)
Second value:  (24.8526802, 67.0348212)
First value:  (24.8526802, 67.0348212)
New Mid Value:  (24.85262385, 67.0347591)
Second value:  (24.8525675, 67.034697)
First value:  (24.8525675, 67.034697)
New Mid Value:  (24.85251075, 67.03460285)
Second value:  (24.852454, 67.0345087)
First value:  (24.852454, 67.0345087)
New Mid Value:  (24.85242435, 67.03446925)
Second value:  (24.8523947, 67.0344298)
First value:  (24.8523947, 67.0344298)
New Mid Value:  (24.85226755, 67.03413535)
Second value:  (24.8521404, 67.0338409)
First value:  (24.8521404, 67.0338409)
New Mid Value:  (24.85199575, 67.0335073)
Second value:  (24.8518511, 67.0331737)
First value:  (24.8518511, 67

Second value:  (24.8772544, 67.0433449)
First value:  (24.8772544, 67.0433449)
New Mid Value:  (24.87730825, 67.0433768)
Second value:  (24.8773621, 67.0434087)
First value:  (24.8773621, 67.0434087)
New Mid Value:  (24.8774102, 67.04346534999999)
Second value:  (24.8774583, 67.043522)
First value:  (24.8774583, 67.043522)
New Mid Value:  (24.87748225, 67.04357815)
Second value:  (24.8775062, 67.0436343)
First value:  (24.8775062, 67.0436343)
New Mid Value:  (24.877515250000002, 67.04369575)
Second value:  (24.8775243, 67.0437572)
First value:  (24.8775243, 67.0437572)
New Mid Value:  (24.877515950000003, 67.04381875)
Second value:  (24.8775076, 67.0438803)
First value:  (24.8775076, 67.0438803)
New Mid Value:  (24.8774953, 67.04392200000001)
Second value:  (24.877483, 67.0439637)
First value:  (24.877483, 67.0439637)
New Mid Value:  (24.877556300000002, 67.04422255)
Second value:  (24.8776296, 67.0444814)
First value:  (24.8776296, 67.0444814)
New Mid Value:  (24.8778437, 67.044824699

First value:  (24.9185598, 67.0979746)
New Mid Value:  (24.91893365, 67.09848364999999)
Second value:  (24.9193075, 67.0989927)
First value:  (24.9193075, 67.0989927)
New Mid Value:  (24.919446049999998, 67.09918139999999)
Second value:  (24.9195846, 67.0993701)
First value:  (24.9195846, 67.0993701)
New Mid Value:  (24.92013805, 67.10005025000001)
Second value:  (24.9206915, 67.1007304)
First value:  (24.9206915, 67.1007304)
New Mid Value:  (24.9207533, 67.10081295)
Second value:  (24.9208151, 67.1008955)
First value:  (24.9208151, 67.1008955)
New Mid Value:  (24.9214154, 67.10167394999999)
Second value:  (24.9220157, 67.1024524)
First value:  (24.9220157, 67.1024524)
New Mid Value:  (24.9222097, 67.10269685)
Second value:  (24.9224037, 67.1029413)
First value:  (24.9224037, 67.1029413)
New Mid Value:  (24.922778649999998, 67.10341825)
Second value:  (24.9231536, 67.1038952)
First value:  (24.9231536, 67.1038952)
New Mid Value:  (24.92329135, 67.10410350000001)
Second value:  (24.9234

First value:  (24.9471555, 67.1819253)
New Mid Value:  (24.947519749999998, 67.18236304999999)
Second value:  (24.947884, 67.1828008)
First value:  (24.947884, 67.1828008)
New Mid Value:  (24.9479855, 67.18291740000001)
Second value:  (24.948087, 67.183034)
First value:  (24.948087, 67.183034)
New Mid Value:  (24.94814775, 67.18312265)
Second value:  (24.9482085, 67.1832113)
First value:  (24.9482085, 67.1832113)
New Mid Value:  (24.948271650000002, 67.18332480000001)
Second value:  (24.9483348, 67.1834383)
First value:  (24.9483348, 67.1834383)
New Mid Value:  (24.94910005, 67.1843494)
Second value:  (24.9498653, 67.1852605)
First value:  (24.9498653, 67.1852605)
New Mid Value:  (24.94993105, 67.1853428)
Second value:  (24.9499968, 67.1854251)
First value:  (24.9499968, 67.1854251)
New Mid Value:  (24.950024900000003, 67.18547290000001)
Second value:  (24.950053, 67.1855207)
First value:  (24.950053, 67.1855207)
New Mid Value:  (24.95006825, 67.1855711)
Second value:  (24.9500835, 67.

...

Second value:  (25.3037236, 68.0758991)
First value:  (25.3037236, 68.0758991)
New Mid Value:  (25.30405595, 68.07662085)
Second value:  (25.3043883, 68.0773426)
First value:  (25.3043883, 68.0773426)
New Mid Value:  (25.304716749999997, 68.0780793)
Second value:  (25.3050452, 68.078816)
First value:  (25.3050452, 68.078816)
New Mid Value:  (25.3054289, 68.07965709999999)
Second value:  (25.3058126, 68.0804982)
First value:  (25.3058126, 68.0804982)
New Mid Value:  (25.3062539, 68.08148875)
Second value:  (25.3066952, 68.0824793)
First value:  (25.3066952, 68.0824793)
New Mid Value:  (25.3070387, 68.08322095)
Second value:  (25.3073822, 68.0839626)
First value:  (25.3073822, 68.0839626)
New Mid Value:  (25.30779875, 68.0849113)
Second value:  (25.3082153, 68.08586)
First value:  (25.3082153, 68.08586)
New Mid Value:  (25.3084904, 68.08645824999999)
Second value:  (25.3087655, 68.0870565)
First value:  (25.3087655, 68.0870565)
New Mid Value:  (25.309049950000002, 68.08768375)
Second val

First value:  (25.4012441, 68.2811084)
New Mid Value:  (25.401246399999998, 68.2811189)
Second value:  (25.4012487, 68.2811294)
First value:  (25.4012487, 68.2811294)
New Mid Value:  (25.4013603, 68.28159525000001)
Second value:  (25.4014719, 68.2820611)
First value:  (25.4014719, 68.2820611)
New Mid Value:  (25.40178915, 68.2832503)
Second value:  (25.4021064, 68.2844395)
First value:  (25.4021064, 68.2844395)
New Mid Value:  (25.4021505, 68.28460405000001)
Second value:  (25.4021946, 68.2847686)
First value:  (25.4021946, 68.2847686)
New Mid Value:  (25.4025958, 68.28626045)
Second value:  (25.402997, 68.2877523)
First value:  (25.402997, 68.2877523)
New Mid Value:  (25.40353845, 68.2896844)
Second value:  (25.4040799, 68.2916165)
First value:  (25.4040799, 68.2916165)
New Mid Value:  (25.40443325, 68.29293725)
Second value:  (25.4047866, 68.294258)
First value:  (25.4047866, 68.294258)
New Mid Value:  (25.4049566, 68.29487635000001)
Second value:  (25.4051266, 68.2954947)
First valu

First value:  (25.4055018, 68.3566426)
New Mid Value:  (25.4054451, 68.3565872)
Second value:  (25.4053884, 68.3565318)
First value:  (25.4053884, 68.3565318)
New Mid Value:  (25.40535895, 68.35652025)
Second value:  (25.4053295, 68.3565087)
First value:  (25.4053295, 68.3565087)
New Mid Value:  (25.40527865, 68.35651415000001)
Second value:  (25.4052278, 68.3565196)
First value:  (25.4052278, 68.3565196)
New Mid Value:  (25.40517285, 68.3565325)
Second value:  (25.4051179, 68.3565454)
First value:  (25.4051179, 68.3565454)
New Mid Value:  (25.40506065, 68.35656725)
Second value:  (25.4050034, 68.3565891)
First value:  (25.4050034, 68.3565891)
New Mid Value:  (25.40494575, 68.35662105)
Second value:  (25.4048881, 68.356653)
First value:  (25.4048881, 68.356653)
New Mid Value:  (25.40482215, 68.35669429999999)
Second value:  (25.4047562, 68.3567356)
First value:  (25.4047562, 68.3567356)
New Mid Value:  (25.4046218, 68.35678909999999)
Second value:  (25.4044874, 68.3568426)
First value:

C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
C:\ProgramData\Anaconda3\envs\ox\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


First value:  (25.3989422, 68.3595994)
New Mid Value:  (25.3993235, 68.35922285)
Second value:  (25.3997048, 68.3588463)
First value:  (25.3997048, 68.3588463)
New Mid Value:  (25.39986955, 68.35867999999999)
Second value:  (25.4000343, 68.3585137)
First value:  (25.4000343, 68.3585137)
New Mid Value:  (25.40018935, 68.3584225)
Second value:  (25.4003444, 68.3583313)
First value:  (25.4003444, 68.3583313)
New Mid Value:  (25.400669100000002, 68.35818645)
Second value:  (25.4009938, 68.3580416)
First value:  (25.4009938, 68.3580416)
New Mid Value:  (25.4011731, 68.35798260000001)
Second value:  (25.4013524, 68.3579236)
First value:  (25.4013524, 68.3579236)
New Mid Value:  (25.40175945, 68.35785920000001)
Second value:  (25.4021665, 68.3577948)
First value:  (25.4021665, 68.3577948)
New Mid Value:  (25.40256965, 68.35770364999999)
Second value:  (25.4029728, 68.3576125)
First value:  (25.4029728, 68.3576125)
New Mid Value:  (25.40341145, 68.3573842)
Second value:  (25.4038501, 68.357155

In [20]:
basic_map = folium.Map(location= (26.277998, 68.288195),
                    zoom_start=7)

folium.TileLayer('Stamen Terrain').add_to(basic_map)
folium.TileLayer('Stamen Toner').add_to(basic_map)
folium.TileLayer('Stamen Water Color').add_to(basic_map)
folium.TileLayer('cartodbpositron').add_to(basic_map)
folium.TileLayer('cartodbdark_matter').add_to(basic_map)
folium.LayerControl().add_to(basic_map)

user_time = str(input('Enter Time for Activity XXXX Hrs: '))

user_dist = get_input_distance(start_time, user_time, cvy_inputs['Speed'])

# calculate distances between coordinates

for index, rows in cvy_inputs.iterrows():
    # get distances based on coordinates and append to dataframe as new column 'Distance'
    dist = get_dist_coordinates(rows['Coordinates'])
    # insert list of distances at a single index in dataframe
    cvy_inputs.at[index, 'Distance'] = dist

# find shortest distance of user_distance - rows['Distance']

for index, rows in cvy_inputs.iterrows():
    relative_dist = get_relative_distance(user_dist[index], rows['Distance'])
    cvy_inputs.at[index, 'Relative Distance'] = relative_dist
    

for index, rows in cvy_inputs.iterrows():
    
    num_vehicles = range(rows['Order'])
    mid_pt = round(len(num_vehicles)/2)
    
    temp_dist = 0
    
    for vehicle in num_vehicles:
        
        index_min = np.argmin(rows['Relative Distance'])
        coordinates_min = rows['Coordinates'][index_min - vehicle]
        
        if vehicle == mid_pt:
            folium.Circle(location= coordinates_min,
                                radius=100*len(num_vehicles)/2,
                                fill_color= rows['Color']).add_to(basic_map)
        
        popup_text = str(rows['Name']) + ' Current Location: ' + str(user_dist[index]) + " KM"


        folium.Marker(location= coordinates_min, 
                      popup= popup_text,
                      icon= folium.Icon(color=rows['Color'], 
                                        icon_color='white', 
                                        icon = 'fa-truck', 
                                        prefix='fa')
                     ).add_to(basic_map)
    
        
#         folium.RegularPolygonMarker(location= coordinates_min 
#                                 , fill_color= rows['Color']
#                                 , number_of_sides=3
#                                 , radius=15
#                                 , rotation=45).add_to(basic_map)
        
        
        create_path(rows['Coordinates'], rows['Start'], rows['Destination'], rows['Color'], basic_map)
    

basic_map

Enter Time for Activity XXXX Hrs: 1130
